<a href="https://colab.research.google.com/github/dungwoong/NN/blob/main/MNIST_with_numpy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Numpy for calculations

In [ ]:
import numpy as np
from numpy.random import randn

# Module classes
Node classes are more like modules. They all implement a forward function that returns outputs of a forward pass and a backwards function that computes gradients

I decided to store x, ans inside the node for use when computing gradients. ```input_grad``` is the gradient with respect to the inputs to each module.

Gradients with respect to module parameters are returned by the backwards() function.

Gradients are not accumulated, they are replaced with each backwards() call, thus zero_grad is not required.

<br><br>

**Difference to existing libraries:**

Existing libraries automatically build a computation graph.
For my implementation, we must manually build a computation graph and perform topological sorting in order to backpropagate correctly. This is undesirable, but is suitable for my purpose of demonstrating backprop.

<br>

In addition, the computation graph is modular, and is thus very simple. Most libraries follow tensor operations to build a computation graph, and allow us to run a backwards() function on the result of any computation to compute gradients. I opted for simpler functionality for my problem.

In [ ]:
# the backprop classes will only support having 1 child, for a simple linear network
# normally you have multiple children, and aggregate the gradient signals from them.

class Node:
  """
  Base class for a single node in the computation graph.

  Acts more like a module. Nodes only have 1 child, so the computation graph
  isn't really a graph.

  Nodes have forwards and backwards methods implemented.
  """
  def __init__(self):
    self.ans, self.x = None, None
    self.input_grad = 0
    self.parameters = []
    self.child = None

  def forward(self, x):
    raise NotImplementedError

  def backwards(self, g):
    # must save gradients WRT inputs in self.input_grad, return a list with same length as self.parameters with gradients for each respective parameter
    raise NotImplementedError

  def __call__(self, *args, **kwargs):
    return self.forward(*args, **kwargs)

## Linear layer

Performs an affine transformation.

In [ ]:
class LinearLayer(Node):
  """
  Linear layer, goes from input_size to output_size.
  """
  def __init__(self, input_size, output_size, bias=False, init_func=randn):
    super().__init__()
    self.w = init_func(output_size, input_size)
    self.b = init_func(output_size, 1) if bias else None
    self.parameters = [self.w, self.b] if bias else [self.w]

  def kaiming_init(self):
    """
    Kaiming initialization
    """
    factor = np.sqrt(2 / self.w.shape[1])
    self.w = randn(*self.w.shape) * factor
    # self.b = randn(*self.b.shape) * factor
    self.b = np.zeros(self.b.shape) if self.b is not None else None
  
  def forward(self, x):
    """
    Calculates Wx + b. Thus, x is input_size x batch_size
    """
    self.x = x
    self.ans = self.w @ x
    if self.b is not None:
      self.ans += self.b
    return self.ans

  def backwards(self, g):
    self.input_grad = self.w.T @ g

    b_grad = [g @ np.ones((g.shape[1], 1))] if self.b is not None else []
    return [g @ self.x.T] + b_grad

## Other modules

These are other modules that will come in handy sometime or another. Implemented the same way as the other modules.

We have:

 - ReLU(rectified linear unit): $max(0, x)$
 - MSE(mean squared error): $\frac{1}{2}||y - t||_2^2$

In [ ]:
class ReLU(Node):
  """
  Rectified linear unit
  """
  def __init__(self):
    super().__init__()

  def forward(self, x):
    self.x = x
    self.ans = np.maximum(0, x)
    return self.ans

  def backwards(self, g):
    jacobian = np.where(self.x > 0, 1, 0)
    self.input_grad = jacobian * self.ans


class MSE(Node):
  """
  Mean squared error
  """
  def __init__(self):
    super().__init__()
    self.t = None
  
  def forward(self, x, t):
    self.x = x
    self.t = t
    self.ans = 0.5 * np.sum((self.x - self.t)**2)
    return self.ans

  def backwards(self, g):
    self.input_grad = self.x - self.t


## Backpropagate function

This function will pass back gradient signals along a list of topologically sorted nodes to perform backprop.

In [ ]:
def back_propagate(top_sorted_nodes):
  """
  Performs backprop.

  Stores output in a dict in the format {node: <grads>}
  where <grads> is a list of grads the same length as the number of parameters in the node.
  """
  grads = dict()
  for node in top_sorted_nodes:
    in_grad = node.child.input_grad if node.child is not None else 1
    g = node.backwards(in_grad)
    if g is not None:
      grads[node] = g
  return grads

## Optimizer and dataset

I used a basic SGD optimizer.

In [ ]:
class SGD:
  """
  SGD optimizer

  x' = x - lr * dL/dx
  """
  def __init__(self, lr):
    self.lr = lr

  def step(self, grads):
    for node in grads:
      for i in range(len(node.parameters)):
        node.parameters[i] -= self.lr * grads[node][i]


class Dataset:
  """
  Dataset class, can quickly generate a batched iterator.
  """
  def __init__(self, inputs, targets):
    self.inputs = inputs
    self.targets = targets
    self.dataset_size = inputs.shape[0]

  def get_batches(self, batch_size):
    num_batches = int(np.ceil(self.dataset_size / batch_size))
    return num_batches

  def batch_iterator(self, batch_size):
    indices = np.arange(self.dataset_size)
    np.random.shuffle(indices)  # Shuffle the indices
    num_batches = int(np.ceil(self.dataset_size / batch_size))

    for batch_num in range(num_batches):
        start_index = batch_num * batch_size
        end_index = min((batch_num + 1) * batch_size, self.dataset_size)
        batch_indices = indices[start_index:end_index]
        yield self.inputs[batch_indices], self.targets[batch_indices]

# Training a simple linear regression model

In [ ]:
# Config params
np.random.seed(42)
dataset_size = 2000
batch_size = 64
slope = np.array([3., 4., 2.]).reshape(3, 1)
intercept = 5
noise_level = 0.5
nepochs = 10

# Setting up the dataset
x = randn(dataset_size, 3)
y = (x @ slope) + intercept + (randn(dataset_size, 1) * noise_level)
data = Dataset(x, y)

# Setting up the modules and computation graph as linear --> MSE
linear = LinearLayer(input_size=3, output_size=1, bias=True)
mse = MSE()
linear.child = mse
nodes = [mse, linear]

optimizer = SGD(lr=0.001)

# Training
print(f"Targets are w={slope}, b={intercept}")
print(f"{data.get_batches(batch_size)} batches in dataset of size {dataset_size}")

for epoch in range(nepochs):
  for x, t in data.batch_iterator(batch_size):
    # x is batch_size x 1, want to change to 1 x batch_size by transposing
    y = linear(x.T)
    loss = mse(y, t.T)

    grads = back_propagate(nodes)
    optimizer.step(grads)
  print(f"Epoch {epoch+1}: W={linear.w[0]}, b={linear.b[0]}, Loss={loss}")

# print(f"Final: W={linear.w[0]}, b={linear.b[0]}, Loss={loss}")

Targets are w=[[3.]
 [4.]
 [2.]], b=5
32 batches in dataset of size 2000
Epoch 1: W=[2.51916934 3.34408372 1.73531408], b=[4.42896598], Loss=8.759483008415106
Epoch 2: W=[2.91275733 3.91033482 1.97277158], b=[4.92334944], Loss=1.4365836037756925
Epoch 3: W=[2.9763836  3.99070275 2.00380442], b=[4.97975813], Loss=1.713785224365796
Epoch 4: W=[2.986183   3.99928556 1.99800597], b=[4.99938726], Loss=3.424956772394891
Epoch 5: W=[2.98992147 3.99752928 1.99911202], b=[4.99348796], Loss=2.603850806885471
Epoch 6: W=[2.97929336 3.99784    2.00269223], b=[5.0024308], Loss=2.3286014020096886
Epoch 7: W=[2.98527337 4.01063784 2.01272288], b=[4.99269835], Loss=2.384166452896827
Epoch 8: W=[2.98568039 4.01872176 2.00181709], b=[4.98184452], Loss=2.1887163308641036
Epoch 9: W=[2.97291501 4.00818726 2.00308851], b=[4.99412349], Loss=1.8316835790255352
Epoch 10: W=[2.98202304 3.99832889 1.99933981], b=[4.99501809], Loss=2.486623610024849


# MNIST

todo figure out chatGPT softmax derivative

https://towardsdatascience.com/derivative-of-the-softmax-function-and-the-categorical-cross-entropy-loss-ffceefc081d1

In [ ]:
class Softmax(Node):
  # by default, inputs should be input_size x batch_size, so we softmax along dim 0

  def forward(self, x):
    # TESTED
    # x should be input_size x batch_size
    self.x = x
    x = np.exp(x)
    sums = np.sum(x, axis=0, keepdims=True)
    self.ans = x / sums
    return self.ans

  def backwards1(self, g):
    # my backwards() implementation
    out = []
    # ans will be input_size x batch_size, we wanna end up with input_size x 1 vectors
    for i in range(self.ans.shape[1]): # go thru obs
      obs = np.expand_dims(self.ans[:, i], 1)
      grad = np.expand_dims(g[:, i], 1)
      m = -obs @ obs.T # input_size x input_size
      np.fill_diagonal(m, obs * (1 - obs))
      out.append(m.T @ grad) # input_size x 1
    # output should be input_size x batch_size
    self.input_grad = np.stack(out).reshape(self.ans.shape)

  # more efficient implementation, by ChatGPT
  # I don't even think this derivative is correct, it might just be proportional
  # pretty sure this is the derivative for sigmoid...
  def backwards2(self, g):
    batch_size = self.x.shape[1]
    dx = self.ans * (g - np.sum(self.ans * g, axis=0, keepdims=True))
    self.input_grad = dx

  def backwards(self, g):
    self.backwards2(g)


class NLLLoss(Node):
  def __init__(self):
    super().__init__()
    self.t = None

  def forward(self, x, t):
    # x, t are num_classes x batch_size
    self.x = x
    self.t = one_hot_encode(t)
    l = -np.log(x)
    self.ans = np.trace(self.t.T @ l) # b x b
    return self.ans
  
  def backwards(self, g=1):
    self.input_grad = -np.reciprocal(self.x) * self.t


def one_hot_encode(x, num_classes=10):
  """
  One hot encodes x if x is a (size,) vector of class labels.
  """
  if len(x.shape) != 1:
    return x
  y = np.zeros((num_classes, x.shape[0]))
  y[x, np.arange(x.shape[0])] = 1
  return y


# Testing if gradients work(DELETE LATER)

softmax + NLL seems to work.

In [ ]:
loss = NLLLoss()
softmax = Softmax()
softmax.child = loss

targets = np.array([[0, 1], [1, 0], [0, 1]]).T
inputs = np.array([[10, -10], [-10, 10], [10, -10]], dtype=float).T

module = [loss, softmax]

for i in range(500):
  logits = softmax(inputs)
  l = loss(logits, targets)
  grads = back_propagate(module)
  inputs -= softmax.input_grad
  if i % 100 == 0:
    print(f"{i}: loss={l}")
    print("Logits:", logits.T.tolist())

0: loss=60.000000006183456
Logits: [[0.9999999979388463, 2.0611536181902033e-09], [2.0611536181902033e-09, 0.9999999979388463], [0.9999999979388463, 2.0611536181902033e-09]]
100: loss=0.016601453438203247
Logits: [[0.005518534447723954, 0.9944814655522761], [0.9944814655522761, 0.005518534447723954], [0.005518534447723954, 0.9944814655522761]]
200: loss=0.007880089523236312
Logits: [[0.0026232497589882965, 0.9973767502410117], [0.9973767502410117, 0.0026232497589882965], [0.0026232497589882965, 0.9973767502410117]]
300: loss=0.005166124417087274
Logits: [[0.0017205596096779036, 0.9982794403903221], [0.9982794403903221, 0.0017205596096779036], [0.0017205596096779036, 0.9982794403903221]]
400: loss=0.0038426757489169313
Logits: [[0.001280071924399478, 0.9987199280756006], [0.9987199280756006, 0.001280071924399478], [0.001280071924399478, 0.9987199280756006]]


In [ ]:
# checking if the two backwards methods calculate the same gradient
loss = NLLLoss()
softmax = Softmax()

inputs = np.array([[10, -10], [-10, 10], [10, -10]], dtype=float).T
g = np.ones((2, 3))
softmax(inputs)
softmax.backwards1(g)
grad1 = np.copy(softmax.input_grad)
softmax(inputs)
softmax.backwards2(g)
grad2 = np.copy(softmax.input_grad)
grad2 - grad1

array([[ 3.70447558e-17, -1.84756286e-25,  1.11022302e-16],
       [-7.39775462e-17,  3.70447558e-17, -1.84756286e-25]])

# MNIST Dataset

In [ ]:
# take the dataset from pytorch, convert to numpy

import torch
from torchvision import datasets
import torchvision.transforms as transforms

# Set the random seed for reproducibility
torch.manual_seed(42)

# Define the transformation to apply to each image
transform = transforms.ToTensor()

# Download and load the MNIST dataset
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)

# Convert the PyTorch datasets to NumPy arrays
train_images = train_dataset.data.numpy()
train_labels = train_dataset.targets.numpy()
test_images = test_dataset.data.numpy()
test_labels = test_dataset.targets.numpy()

train_images = train_images / 255
test_images = test_images / 255

# Print the shape of the arrays
print("Train images shape:", train_images.shape)
print("Train labels shape:", train_labels.shape)
print("Test images shape:", test_images.shape)
print("Test labels shape:", test_labels.shape)

100%|██████████| 9912422/9912422 [00:00<00:00, 82335477.90it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 74867548.72it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 21159933.17it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


100%|██████████| 4542/4542 [00:00<00:00, 3385556.92it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



Train images shape: (60000, 28, 28)
Train labels shape: (60000,)
Test images shape: (10000, 28, 28)
Test labels shape: (10000,)


In [ ]:
def make_sequential(layers):
  layers = layers[0:]
  for i in range(0, len(layers) - 1):
    layers[i].child = layers[i+1]
  layers.reverse()
  return layers

class FCModel:
  def __init__(self, input_shape=784):
    self.softmax = Softmax()
    self.loss = NLLLoss()
    self.layers = [LinearLayer(input_shape, 512, bias=True),
                   ReLU(),
                   LinearLayer(512, 256, bias=True),
                   ReLU(),
                   LinearLayer(256, 10, bias=True),
                   self.softmax]
    self.layers[0].kaiming_init()
    self.layers[2].kaiming_init()

    self.softmax.child = self.loss
    self.topsorted = [self.loss] + make_sequential(self.layers)

  def forward(self, x):
    # x is batch_size x 28 x 28
    x = x.reshape(x.shape[0], -1)
    x = x.T
    for layer in self.layers:
      x = layer(x)
    return x

In [ ]:
def get_accuracy(out, targets):
  # targets are (BS,), out is (num_classes, BS) logits.
  preds = np.argmax(out.T, axis=1)
  # print(preds.shape, targets.shape)
  correct = np.sum(preds == targets)
  return correct, targets.shape[0]

def test_get_accuracy(n=1000):
  targets = np.ones((n,))
  s = Softmax()
  out = randn(10, n)
  out = s(out)
  return get_accuracy(out, targets)

test_get_accuracy() # should be around 0.1

(109, 1000)

In [ ]:
def train_one_epoch(model, train, valid, optimizer, args, label=""):
  # trains for one epoch
  training_losses = []
  valid_losses = []
  valid_t, valid_c = 0, 0

  # training
  print(f"[{label}]Training")
  for x, t in train.batch_iterator(args['batch_size']):
    # forwards
    x = model.forward(x)

    # get loss and backpropagate
    loss = model.loss(x, t)
    training_losses.append(loss)
    grads = back_propagate(model.topsorted)
    optimizer.step(grads)

  # validation
  print(f"[{label}]Evaluating")
  for x, t in valid.batch_iterator(args['batch_size']):
    x = model.forward(x)
    loss = model.loss(x, t)
    valid_losses.append(loss)
    correct, total = get_accuracy(x, t)
    valid_t += total
    valid_c += correct

  tl = sum(training_losses) / len(training_losses)
  vl = sum(valid_losses) / len(valid_losses)
  acc = valid_c / valid_t
  print(f"Training loss {round(tl, 4)}, valid loss {round(vl, 4)}, valid acc {round(acc*100)}%")
  

In [ ]:
train = Dataset(train_images, train_labels)
valid = Dataset(test_images, test_labels)
print(f"train is {train.dataset_size}, valid is {valid.dataset_size}")

optimizer = SGD(lr=1e-3)
model = FCModel()
args = {'batch_size': 64}

for layer in model.layers:
  print(layer)
  if layer.child is None:
    print("no child")

for i in range(10):
  label = f"Epoch {i+1}"
  train_one_epoch(model, train, valid, optimizer, args, label=label)

train is 60000, valid is 10000
[Epoch 1]Training
[Epoch 1]Evaluating
Training loss 160.617, valid loss 81.7236, valid acc 62%
[Epoch 2]Training
[Epoch 2]Evaluating
Training loss 68.2646, valid loss 56.9234, valid acc 73%
[Epoch 3]Training
[Epoch 3]Evaluating
Training loss 53.2787, valid loss 47.9233, valid acc 77%
[Epoch 4]Training
[Epoch 4]Evaluating
Training loss 46.4706, valid loss 42.9524, valid acc 80%
[Epoch 5]Training
[Epoch 5]Evaluating
Training loss 42.3122, valid loss 39.542, valid acc 81%
[Epoch 6]Training
[Epoch 6]Evaluating
Training loss 39.4029, valid loss 37.1218, valid acc 82%
[Epoch 7]Training
[Epoch 7]Evaluating
Training loss 37.1857, valid loss 35.2812, valid acc 83%
[Epoch 8]Training
[Epoch 8]Evaluating
Training loss 35.4567, valid loss 33.7703, valid acc 84%
[Epoch 9]Training
[Epoch 9]Evaluating
Training loss 34.0272, valid loss 32.5093, valid acc 84%
[Epoch 10]Training
[Epoch 10]Evaluating
Training loss 32.8403, valid loss 31.5672, valid acc 85%


Kinda low accuracy, but the main takeaway is that this worked.